In [ ]:
import sys
print("sys.executable:", sys.executable)
print("sys.version:", sys.version)
!{sys.executable} -m pip install numpy pandas seaborn mesa ipykernel


In [ ]:
# Has multi-dimensional arrays and matrices.
# Has a large collection of mathematical functions to operate on these arrays.
import numpy as np

# Data manipulation and analysis.
import pandas as pd

# Data visualization tools.
import seaborn as sns

import mesa

# Import Cell Agent and OrthogonalMooreGrid
from mesa.discrete_space import CellAgent, OrthogonalMooreGrid

import math

/Users/leonibernabe/Documents/TECdeMTY/5thSemester/Agentes/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Instantiate CleanerAgent as CellAgent
class CleanerAgent(CellAgent):
    """An agent with fixed initial wealth."""

    def __init__(self, model, cell):
        super().__init__(model)
        self.cell = cell  # Instantiate agent with location (x,y)
        self.moves = 0

    # Move Function
    def move(self):
      new_cell = self.cell.neighborhood.select_random_cell()
      self.cell = new_cell
      self.moves += 1

    def clean_cell(self):
        if(self.cell.dirty):
          self.cell.dirty=False
        else:
          self.move()

In [ ]:
class CleanningModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, n, width, height, seed=None, percentage_dirty=10.0):
        super().__init__(seed=seed)
        self.num_agents = n
        # Instantiate an instance of Moore neighborhood space
        self.grid = OrthogonalMooreGrid(
            (width, height), torus=False, capacity=n, random=self.random
        )

        for cell in self.grid.all_cells:
          cell.dirty = False

        # selection of random cells to be dirty
        total_cells = width*height
        num_dirty_cells = math.ceil((percentage_dirty / 100) * total_cells)

        for _ in range(num_dirty_cells):
          cell = self.grid.all_cells.select_random_cell()

          while cell.dirty:
            cell = self.grid.all_cells.select_random_cell()

          cell.dirty = True


        # give the initial positions of the agents
        start_cell = self.grid[(0, 0)]
        start_cells = [start_cell] * self.num_agents

        # Create agents
        agents = CleanerAgent.create_agents(
            self,
            self.num_agents,
            start_cells
        )


    def step(self):
        self.agents.shuffle_do("clean_cell")

    def dirty_cells_count(self):
      cnt = 0
      for cell in self.grid.all_cells:
        cnt += cell.dirty

      return cnt

    def total_movements(self):
      cnt = 0
      for agent in self.agents:
        cnt += agent.moves

      return cnt

In [ ]:
import matplotlib.pyplot as plt

def plot_grid(model):
    width, height = model.grid.width, model.grid.height
    grid_data = np.zeros((height, width))

    # Marca las celdas sucias
    for x in range(width):
        for y in range(height):
            cell = model.grid[(x, y)]
            grid_data[y, x] = 1 if cell.dirty else 0

    # Dibuja el grid
    plt.imshow(grid_data, cmap="YlOrBr", origin="lower")
    plt.title("Dirty cells = brown | Clean = white")

    plt.show()

In [ ]:

width = int(input("Ancho del cuarto (M): "))
height = int(input("Alto del cuarto (N): "))
num_agents = int(input("Número de agentes: "))
percentage_dirty = float(input("Porcentaje de celdas sucias iniciales (%): "))
max_steps = int(input("Tiempo máximo de ejecución (pasos): "))

model = CleanningModel(num_agents, width, height, None, percentage_dirty)

total_time = -1

dirty_cells = model.dirty_cells_count()

for i in range(max_steps):
    dirty_cells = model.dirty_cells_count()

    if dirty_cells == 0 and total_time==-1:
      total_time = i+1

    # plot_grid(model)
    model.step()

if total_time == -1:
    total_time = max_steps

total_cells = width * height
clean_cells = total_cells - dirty_cells
percentage_clean = (clean_cells / total_cells) * 100

total_moves = sum(agent.moves for agent in model.agents)

print("\n--- Resultados de la simulación ---")
print(f"Tiempo total hasta limpieza completa (o máximo): {total_time} pasos")
print(f"Porcentaje de celdas limpias: {percentage_clean:.2f}%")
print(f"Movimientos totales realizados por los agentes: {total_moves}")

